# Configure

In [ ]:
#!/usr/bin/env python3
"""
Script to train, evaluate, and compare multiple regression models
(Pooled OLS, Bank FE, Random Forest, XGBoost) on panel data.

Loads pre-split and preprocessed data (X_train, y_train, etc.).
Assumes data is clean and aligned. Trains, tunes (optional),
predicts, evaluates, and saves models using simplified functions,
but ensures constant is handled correctly for OLS.
"""

%pip install scikit-learn --quiet

import os
import time
import warnings
from datetime import datetime
import joblib
import json
import numpy as np
import pandas as pd
import statsmodels.api as sm
import xgboost as xgb
from linearmodels.panel import PanelOLS
from scipy.stats import randint, uniform
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import (mean_absolute_error, mean_squared_error,
                             r2_score)
from sklearn.model_selection import GroupKFold, RandomizedSearchCV
import traceback # Keep for debugging potential errors

# =============================================================================
# Configuration
# =============================================================================

# --- File Paths ---
DATA_PATHS = {
    'X_train': 'X_train.parquet', 'y_train': 'y_train.parquet',
    'X_val':   'X_val.parquet',   'y_val':   'y_val.parquet',
    'X_test':  'X_test.parquet',  'y_test':  'y_test.parquet',
}
SAVE_DIR = 'models'
# --- Data Structure ---
ENTITY_ID_COL = 'id'
TIME_COL = 'date'
# --- Model Selection ---
MODELS_TO_TRAIN = {
    'Pooled OLS': True,
    'Bank FE': True,
    'Random Forest': True,
    'XGBoost': True,
}
# --- Tuning Configuration ---
DO_TUNING = True
N_ITER_RANDOM_SEARCH = 100
CV_FOLDS = 5
SCORING_METRIC = 'neg_root_mean_squared_error'
# --- Default Model Parameters ---
RF_DEFAULT_PARAMS = { 'n_estimators': 200, 'max_depth': 30, 'min_samples_leaf': 10, 'n_jobs': -1, 'random_state': 42 }

# Best XGBoost Parameters found: {'colsample_bytree': 0.8822634555704314, 'gamma': 0.08526206184364576, 'learning_rate': 0.023010318597055907, 'max_depth': 6, 'n_estimators': 188, 'reg_alpha': 0.9656320330745594, 'reg_lambda': 0.8083973481164611, 'subsample': 0.7913841307520112}
  
XGB_DEFAULT_PARAMS = { 'n_estimators': 200, 'learning_rate': 0.1,   'max_depth': 7, 'subsample': 0.8, 'colsample_bytree': 0.8, 'objective': 'reg:squarederror', 'n_jobs': -1, 'random_state': 42, 'early_stopping_rounds': 10, 'tree_method': 'hist' }
XGB_DEFAULT_PARAMS = { 'n_estimators': 188, 'learning_rate': 0.023, 'max_depth': 6, 'subsample': 0.9, 'colsample_bytree': 0.8, 'objective': 'reg:squarederror', 'n_jobs': -1, 'random_state': 42, 'early_stopping_rounds': 10, 'tree_method': 'hist' }

# --- Random Search Parameter Distributions ---
RF_PARAM_DIST = { 'n_estimators': randint(100, 500), 'max_depth': randint(10, 50), 'min_samples_split': randint(2, 20), 'min_samples_leaf': randint(5, 30), 'max_features': uniform(0.6, 0.4) }
XGB_PARAM_DIST = { 'n_estimators': randint(100, 600), 'learning_rate': uniform(0.01, 0.2), 'max_depth': randint(3, 12), 'subsample': uniform(0.7, 0.3), 'colsample_bytree': uniform(0.7, 0.3), 'gamma': uniform(0, 0.5), 'reg_alpha': uniform(0, 1), 'reg_lambda': uniform(0, 1) }



# Functions

In [2]:

# =============================================================================
# Helper Functions
# =============================================================================

def check_and_convert_index(df, df_name):
    """Minimal check: Validates MultiIndex structure and ensures time level is Timestamp."""
    print(f"Checking index for {df_name}...")
    if df is None or df.empty: return df
    if not isinstance(df.index, pd.MultiIndex): return df
    expected_names = [ENTITY_ID_COL, TIME_COL]
    if list(df.index.names) != expected_names: warnings.warn(f"{df_name} index names {df.index.names}, expected {expected_names}.")
    try:
        time_level = df.index.get_level_values(TIME_COL)
        if pd.api.types.is_period_dtype(time_level.dtype):
            print(f"  Converting PeriodDtype index in {df_name} to Timestamp...")
            entity_level = df.index.get_level_values(ENTITY_ID_COL)
            new_time_level = time_level.to_timestamp()
            df.index = pd.MultiIndex.from_arrays([entity_level, new_time_level], names=expected_names)
            df.sort_index(inplace=True)
        elif not pd.api.types.is_datetime64_any_dtype(time_level.dtype):
             print(f"  Warning: Non-standard time index in {df_name}. Attempting conversion.")
             entity_level = df.index.get_level_values(ENTITY_ID_COL)
             new_time_level = pd.to_datetime(time_level.to_series(keep_tz=True))
             df.index = pd.MultiIndex.from_arrays([entity_level, new_time_level], names=expected_names)
             df.sort_index(inplace=True)
    except Exception as e: print(f"ERROR processing time index for {df_name}: {e}"); raise
    return df

def evaluate_predictions(y_true, y_pred, model_name, data_set_name):
    """Calculates key regression metrics. Assumes aligned, non-empty inputs."""
    print(f"  Evaluating {model_name} on {data_set_name} set...")
    if y_pred is None: # Keep basic check for valid predictions
        print("    Skipping evaluation: No predictions provided.")
        return {'R2': np.nan, 'MSE': np.nan, 'RMSE': np.nan, 'MAE': np.nan}
    y_true_aligned, y_pred_aligned = y_true.align(y_pred, join='inner', copy=False)
    mask = y_true_aligned.notna() & y_pred_aligned.notna()
    y_true_eval = y_true_aligned[mask]
    y_pred_eval = y_pred_aligned[mask]
    if y_true_eval.empty:
        print("    Skipping evaluation: No valid aligned pairs after dropping NaNs.")
        return {'R2': np.nan, 'MSE': np.nan, 'RMSE': np.nan, 'MAE': np.nan}
    mse = mean_squared_error(y_true_eval, y_pred_eval)
    mae = mean_absolute_error(y_true_eval, y_pred_eval)
    r2 = r2_score(y_true_eval, y_pred_eval)
    rmse = np.sqrt(mse)
    print(f"    R2: {r2:.4f}, RMSE: {rmse:.4f}, MAE: {mae:.4f} ({len(y_true_eval)} obs)")
    return {'R2': r2, 'MSE': mse, 'RMSE': rmse, 'MAE': mae}

# display_results and save_model_artifacts remain the same
def display_results(evaluation_results):
    """Consolidates evaluation results into a DataFrame and prints a summary table."""
    print("\n" + "="*40); print(" Consolidated Evaluation Results "); print("="*40)
    results_list = []
    for model, datasets_metrics in evaluation_results.items():
        for dataset, metrics in datasets_metrics.items():
            row = metrics.copy(); row['Model'] = model; row['Dataset'] = dataset
            results_list.append(row)
    if not results_list: print("No evaluation results."); return None
    results_df = pd.DataFrame(results_list)
    cols_order = ['Model', 'Dataset', 'R2', 'RMSE', 'MAE', 'MSE']
    available_cols = [col for col in cols_order if col in results_df.columns]
    results_df = results_df[available_cols]
    try:
        numeric_metrics = ['R2', 'RMSE', 'MAE', 'MSE']
        value_cols = [m for m in numeric_metrics if m in results_df.columns]
        if value_cols and 'Model' in results_df.columns and 'Dataset' in results_df.columns:
            results_agg = results_df.groupby(['Model', 'Dataset'], observed=False, as_index=False)[value_cols].mean()
            results_pivot = results_agg.pivot(index='Model', columns='Dataset', values=value_cols)
            results_pivot.columns = ['_'.join(map(str, col)).strip() for col in results_pivot.columns.values]
            try: # Attempt sorting
                dataset_order = pd.CategoricalDtype(['train', 'val', 'test'], ordered=True)
                metric_order = pd.CategoricalDtype(['R2', 'RMSE', 'MAE', 'MSE'], ordered=True)
                col_metrics = results_pivot.columns.str.split('_').str[0]
                col_datasets = results_pivot.columns.str.split('_').str[-1]
                col_metrics_cat = pd.Categorical(col_metrics, categories=metric_order.categories, ordered=True)
                col_datasets_cat = pd.Categorical(col_datasets, categories=dataset_order.categories, ordered=True)
                sort_indices = np.lexsort((col_datasets_cat.codes, col_metrics_cat.codes))
                results_pivot = results_pivot.iloc[:, sort_indices]
            except Exception as sort_e: print(f"  Warning: Could not sort pivot table cols ({sort_e}).")
            print("\n--- Performance Summary (Metrics by Dataset) ---")
            with pd.option_context('display.float_format', '{:.4f}'.format): print(results_pivot)
            return results_pivot
        else: # Fallback display
            print("\nNo numeric metrics or pivot columns. Raw results:")
            with pd.option_context('display.float_format', '{:.4f}'.format): print(results_df)
            return results_df
    except Exception as e: # Fallback display on pivot error
        print("\nCould not create pivot table. Raw results:")
        with pd.option_context('display.float_format', '{:.4f}'.format): print(results_df); print(f"(Pivot error: {e})")
        return results_df

def save_model_artifacts(model, model_name, best_params, save_dir, timestamp_prefix):
    """Saves model and parameters (simplified error handling)."""
    if model is None: return
    print(f"\n--- Saving Artifacts for {model_name} ---")
    os.makedirs(save_dir, exist_ok=True)
    model_filename_base = model_name.replace(' ', '_').lower()
    model_filename = f"{timestamp_prefix}_{model_filename_base}_model.joblib"
    save_path = os.path.join(save_dir, model_filename)
    joblib.dump(model, save_path)
    print(f"  Saved model to '{save_path}'")
    if best_params:
         param_filename = f"{timestamp_prefix}_{model_filename_base}_best_params.json"
         param_save_path = os.path.join(save_dir, param_filename)
         try:
             serializable_params = {k: (int(v) if isinstance(v, np.integer) else float(v) if isinstance(v, np.floating) else v) for k, v in best_params.items()}
             with open(param_save_path, 'w') as f: json.dump(serializable_params, f, indent=4)
             print(f"  Saved best parameters to '{param_save_path}'")
         except Exception as e_p: print(f"  ERROR saving best parameters for {model_name}: {e_p}")

# =============================================================================
# Model Training Functions
# =============================================================================

def train_pooled_ols(X_train, y_train):
    """Trains Pooled OLS. Assumes X_train has features, y_train is target Series. Adds constant."""
    print("\n--- Training Pooled OLS ---")
    y_train_ols = y_train.squeeze()
    # Add constant before fitting
    X_train_ols = sm.add_constant(X_train, has_constant='add')
    ols_model = sm.OLS(y_train_ols, X_train_ols)
    ols_results = ols_model.fit()
    print(f"Pooled OLS model fitted using {ols_results.nobs} observations.")
    # Return model and the names of parameters actually fitted (including 'const')
    return ols_results, ols_results.params.index.tolist()

def train_bank_fe(X_train, y_train):
    """Trains Bank Fixed Effects. Assumes clean, aligned X_train, y_train with MultiIndex."""
    print("\n--- Training Bank Fixed Effects (FE) ---")
    y_train_fe = y_train.squeeze()
    fe_model = PanelOLS(y_train_fe, X_train, entity_effects=True, drop_absorbed=True)
    fe_results = fe_model.fit(cov_type='clustered', cluster_entity=True)
    print(f"Bank Fixed Effects model fitted using {fe_results.nobs} observations.")
    effective_features = fe_results.params.index.tolist() # Get features used post-absorption
    absorbed_vars = list(set(X_train.columns) - set(effective_features))
    if absorbed_vars: print(f"  Note: Variables likely absorbed by FE: {absorbed_vars}")
    # Return model and names of parameters fitted (effective features)
    return fe_results, effective_features

# RF and XGBoost training functions 
def train_random_forest(X_train, y_train, entity_col, tune_config):
    """Tunes (optional) and trains Random Forest. Assumes clean, aligned X/y."""
    print("\n--- Training Pooled Random Forest ---")
    y_train_rf = y_train.squeeze().values.ravel()
    groups_rf = X_train.index.get_level_values(entity_col)
    X_train_rf = X_train

    best_params = None
    current_rf_params = RF_DEFAULT_PARAMS.copy()
    rf_model = RandomForestRegressor(**current_rf_params, verbose=0)

    if tune_config['do_tuning']:
        print(f"  Running Randomized Search CV ({tune_config['n_iter']} iterations)...")
        group_kfold = GroupKFold(n_splits=tune_config['cv_folds'])
        base_estimator = RandomForestRegressor(random_state=42, n_jobs=-1)
        rf_random_search = RandomizedSearchCV(
            estimator=base_estimator, param_distributions=RF_PARAM_DIST,
            n_iter=tune_config['n_iter'], cv=group_kfold, scoring=tune_config['scoring_metric'],
            n_jobs=-1, random_state=42, verbose=1, error_score='raise'
        )
        rf_random_search.fit(X_train_rf, y_train_rf, groups=groups_rf)
        print(f"  Best RF Parameters found: {rf_random_search.best_params_}")
        print(f"  Best RF CV Score ({tune_config['scoring_metric']}): {rf_random_search.best_score_:.4f}")
        rf_model = rf_random_search.best_estimator_
        best_params = rf_random_search.best_params_
    else:
        print("  Skipping tuning, using default parameters.")
        rf_model.fit(X_train_rf, y_train_rf)
        print(f"Random Forest fitted with default parameters.")

    if hasattr(rf_model, 'feature_importances_'):
        importances = pd.Series(rf_model.feature_importances_, index=X_train_rf.columns)
        print("\n  --- Random Forest Feature Importances (Top 15) ---")
        print(importances.nlargest(15).to_string())

    # Return model, params, and columns used (all columns from X_train)
    return rf_model, best_params, X_train_rf.columns.tolist()


def train_xgboost(X_train, y_train, entity_col, tune_config, X_val=None, y_val=None):
    """Tunes (optional) and trains XGBoost. Handles NaN targets minimally."""
    print("\n--- Training Pooled XGBoost ---")
    target_col_name = y_train.squeeze().name

    # Minimal cleaning: Join X/y, drop rows with NaN target, split back
    df_train_temp = X_train.join(y_train.squeeze().rename(target_col_name), how='inner')
    df_train_temp.dropna(subset=[target_col_name], inplace=True)
    X_train_xgb_clean = df_train_temp[X_train.columns]
    y_train_xgb_clean = df_train_temp[target_col_name]
    groups_xgb = df_train_temp.index.get_level_values(entity_col)
    del df_train_temp

    if y_train_xgb_clean.empty: print("ERROR: No training data after dropping NaN targets."); return None, None, None

    # Prepare validation set
    eval_set = None
    fit_params = {}
    if X_val is not None and y_val is not None:
        print("  Preparing validation set...")
        df_val_temp = X_val.join(y_val.squeeze().rename(target_col_name), how='inner')
        df_val_temp.dropna(subset=[target_col_name], inplace=True)
        if not df_val_temp.empty:
            X_val_xgb_clean = df_val_temp[X_train.columns] # Ensure same columns as train
            y_val_xgb_clean = df_val_temp[target_col_name]
            eval_set = [(X_train_xgb_clean, y_train_xgb_clean), (X_val_xgb_clean, y_val_xgb_clean)]
            print("  Validation set prepared.")
        else: print("  Validation set empty after dropping NaN targets.")
        del df_val_temp
    else: print("  No validation data provided.")

    best_params = None
    current_xgb_params = XGB_DEFAULT_PARAMS.copy()
    xgb_model = None

    if tune_config['do_tuning']:
        print(f"  Running Randomized Search CV ({tune_config['n_iter']} iterations)...")
        group_kfold = GroupKFold(n_splits=tune_config['cv_folds'])
        xgb_base = xgb.XGBRegressor(objective='reg:squarederror', n_jobs=-1, random_state=42, tree_method=current_xgb_params.get('tree_method', 'auto'))
        xgb_random_search = RandomizedSearchCV(
            estimator=xgb_base, param_distributions=XGB_PARAM_DIST,
            n_iter=tune_config['n_iter'], cv=group_kfold, scoring=tune_config['scoring_metric'],
            n_jobs=-1, random_state=42, verbose=1, error_score='raise'
        )
        xgb_random_search.fit(X_train_xgb_clean, y_train_xgb_clean, groups=groups_xgb)
        print(f"  Best XGBoost Parameters found: {xgb_random_search.best_params_}")
        print(f"  Best XGBoost CV Score ({tune_config['scoring_metric']}): {xgb_random_search.best_score_:.4f}")
        best_params = xgb_random_search.best_params_

        print("  Refitting best model on full training data...")
        final_params = best_params.copy()
        final_params.update({k: v for k, v in current_xgb_params.items() if k in ['tree_method', 'objective', 'random_state', 'n_jobs']})
        fit_final_params = {'verbose': False}
        if eval_set:
            final_params['early_stopping_rounds'] = current_xgb_params.get('early_stopping_rounds', 10)
            fit_final_params['eval_set'] = eval_set
        xgb_model = xgb.XGBRegressor(**final_params)
        xgb_model.fit(X_train_xgb_clean, y_train_xgb_clean, **fit_final_params)
        if eval_set and hasattr(xgb_model, 'best_iteration'):
             print(f"    Refit completed. Best iteration: {xgb_model.best_iteration}")
             best_params['n_estimators'] = xgb_model.best_iteration
        else: print("    Refit completed.")
    else:
        print("  Skipping tuning, using default parameters.")
        xgb_model = xgb.XGBRegressor(**current_xgb_params)
        fit_params_default = {'verbose': False}
        if eval_set: fit_params_default['eval_set'] = eval_set
        xgb_model.fit(X_train_xgb_clean, y_train_xgb_clean, **fit_params_default)
        print(f"XGBoost fitted with default parameters.")
        if eval_set and hasattr(xgb_model, 'best_iteration'): print(f"  Best iteration: {xgb_model.best_iteration}")

    return xgb_model, best_params, X_train.columns.tolist()


# run_predictions modified to accept train_cols again for OLS/FE
def run_predictions(model, model_name, X_data, train_cols=None):
    """
    Generates predictions. Assumes X_data is clean.
    Requires train_cols for OLS/FE to specify columns (incl const/effective features).
    For RF/XGB, assumes X_data contains the correct features.
    """
    print(f"  Generating predictions for {model_name}...")
    if model is None or X_data is None or X_data.empty: return None

    predictions = None
    try:
        if model_name == 'Pooled OLS':
            if train_cols is None: raise ValueError("train_cols (incl const) needed for OLS.")
            # Ensure input X_data has columns from train_cols (excluding const) before adding constant
            feature_cols_ols = [c for c in train_cols if c != 'const']
            X_pred_ols = sm.add_constant(X_data[feature_cols_ols], has_constant='add')
            # Align to the exact columns from training (incl 'const' and correct order)
            X_pred_ols = X_pred_ols.reindex(columns=train_cols, fill_value=np.nan)
            predictions = model.predict(X_pred_ols)

        elif model_name == 'Bank FE':
            if train_cols is None: raise ValueError("train_cols (effective features) needed for FE.")
            # Select only the effective feature columns used in training
            X_pred_fe = X_data[train_cols]
            # Predict using the effective features
            predictions = model.predict(exog=X_pred_fe)
            if isinstance(predictions, pd.DataFrame): predictions = predictions.iloc[:, 0]
            # Reindex to original X_data index (predict might drop rows if NaNs were present)
            # Assuming model.predict doesn't drop rows if input is clean
            predictions = pd.Series(predictions, index=X_pred_fe.index).reindex(X_data.index)

        elif model_name in ['Random Forest', 'XGBoost']:
            # Assumes X_data contains exactly the columns the model was trained on
            # If train_cols were provided (e.g., X_train.columns), ensure X_data matches
            if train_cols and set(X_data.columns) != set(train_cols):
                 warnings.warn(f"Columns in X_data for {model_name} prediction might not match training columns. Ensure order/content is correct.")
                 # Reindex just in case, although ideally X_data is already correct
                 X_data = X_data.reindex(columns=train_cols, fill_value=np.nan)

            predictions = model.predict(X_data)

        # Return as a Series with original index
        if predictions is not None:
            predictions = pd.Series(predictions, index=X_data.index, name=f"{model_name}_pred")
            print(f"  Generated {predictions.notna().sum()} non-NaN {model_name} predictions.")

    except Exception as e:
        print(f"  ERROR during prediction generation for {model_name}: {e}")
        traceback.print_exc()
        return None

    return predictions



# Execute

## Load data

In [3]:


start_time = time.time()
timestamp_prefix = datetime.now().strftime("%y%m%d_%H%M")
print(f"Script started at: {datetime.now()}")
print(f"Saving directory: '{SAVE_DIR}'")
os.makedirs(SAVE_DIR, exist_ok=True)

# --- 1. Load Pre-Split Data & Check Index ---
print("\n--- Loading Pre-Split Datasets & Checking Index ---")
loaded_data = {}
all_data_loaded = True
for name, path in DATA_PATHS.items():
    try:
        df = pd.read_parquet(path)
        df = check_and_convert_index(df, name)
        loaded_data[name] = df
    except Exception as e:
        print(f"CRITICAL ERROR loading or checking index for {name}: {e}")
        all_data_loaded = False; break
if not all_data_loaded: exit()

# Assign loaded data to variables
X_train = loaded_data['X_train']
y_train = loaded_data['y_train']
X_val = loaded_data.get('X_val')
y_val = loaded_data.get('y_val')
X_test = loaded_data.get('X_test')
y_test = loaded_data.get('y_test')


# --- 2. Identify Target Variable Name ---
print("\n--- Identifying Target Variable ---")
target_variable = y_train.columns[0]
print(f"Target variable identified as: '{target_variable}'")
# feature_list is implicitly all columns in X_train



Script started at: 2025-05-17 18:17:50.322046
Saving directory: 'models'

--- Loading Pre-Split Datasets & Checking Index ---
Checking index for X_train...
  Converting PeriodDtype index in X_train to Timestamp...
Checking index for y_train...
  Converting PeriodDtype index in y_train to Timestamp...
Checking index for X_val...
  Converting PeriodDtype index in X_val to Timestamp...
Checking index for y_val...
  Converting PeriodDtype index in y_val to Timestamp...
Checking index for X_test...
  Converting PeriodDtype index in X_test to Timestamp...
Checking index for y_test...
  Converting PeriodDtype index in y_test to Timestamp...

--- Identifying Target Variable ---
Target variable identified as: 'roa'


C:\Users\tansc\AppData\Local\Temp\ipykernel_12184\2146223430.py:14: DeprecationWarning: is_period_dtype is deprecated and will be removed in a future version. Use `isinstance(dtype, pd.PeriodDtype)` instead
  if pd.api.types.is_period_dtype(time_level.dtype):
C:\Users\tansc\AppData\Local\Temp\ipykernel_12184\2146223430.py:14: DeprecationWarning: is_period_dtype is deprecated and will be removed in a future version. Use `isinstance(dtype, pd.PeriodDtype)` instead
  if pd.api.types.is_period_dtype(time_level.dtype):
C:\Users\tansc\AppData\Local\Temp\ipykernel_12184\2146223430.py:14: DeprecationWarning: is_period_dtype is deprecated and will be removed in a future version. Use `isinstance(dtype, pd.PeriodDtype)` instead
  if pd.api.types.is_period_dtype(time_level.dtype):
C:\Users\tansc\AppData\Local\Temp\ipykernel_12184\2146223430.py:14: DeprecationWarning: is_period_dtype is deprecated and will be removed in a future version. Use `isinstance(dtype, pd.PeriodDtype)` instead
  if pd.api.t

## Train models

In [4]:

# --- 4. Train Models ---
print("\n" + "="*30); print(" Model Training "); print("="*30)
trained_models = {}
best_params_all = {}
training_columns = {} # Reintroduced to store columns used by models (esp. OLS/FE)

tune_config_dict = { 'do_tuning': DO_TUNING, 'n_iter': N_ITER_RANDOM_SEARCH, 'cv_folds': CV_FOLDS, 'scoring_metric': SCORING_METRIC }
y_train_series = y_train[target_variable] # Use Series for y

if MODELS_TO_TRAIN.get('Pooled OLS'):
    model_ols, cols_ols = train_pooled_ols(X_train, y_train_series)
    trained_models['Pooled OLS'] = model_ols
    training_columns['Pooled OLS'] = cols_ols # Store fitted columns (incl. const)
    best_params_all['Pooled OLS'] = None

if MODELS_TO_TRAIN.get('Bank FE'):
    model_fe, cols_fe = train_bank_fe(X_train, y_train_series)
    trained_models['Bank FE'] = model_fe
    training_columns['Bank FE'] = cols_fe # Store effective feature columns
    best_params_all['Bank FE'] = None

if MODELS_TO_TRAIN.get('Random Forest'):
    # RF needs original features; training function returns them
    model_rf, params_rf, cols_rf = train_random_forest(X_train, y_train_series, ENTITY_ID_COL, tune_config_dict)
    trained_models['Random Forest'] = model_rf
    best_params_all['Random Forest'] = params_rf
    training_columns['Random Forest'] = cols_rf # Store feature columns used

if MODELS_TO_TRAIN.get('XGBoost'):
    # XGB needs original features; training function returns them
    model_xgb, params_xgb, cols_xgb = train_xgboost(
        X_train, y_train_series, ENTITY_ID_COL, tune_config_dict,
        X_val=X_val, y_val=(y_val[target_variable] if y_val is not None else None)
    )
    trained_models['XGBoost'] = model_xgb
    best_params_all['XGBoost'] = params_xgb
    training_columns['XGBoost'] = cols_xgb # Store feature columns used




 Model Training 

--- Training Pooled OLS ---
Pooled OLS model fitted using 384.0 observations.

--- Training Bank Fixed Effects (FE) ---
Bank Fixed Effects model fitted using 384 observations.

--- Training Pooled Random Forest ---
  Skipping tuning, using default parameters.
Random Forest fitted with default parameters.

  --- Random Forest Feature Importances (Top 15) ---
roa_lag1                      0.728738
trading_assets_ratio_lag1     0.065668
equity_to_asset_ratio_lag1    0.056706
log_total_assets_lag1         0.045614
deposit_ratio_lag1            0.021562
household_delinq_diff_lag1    0.013678
household_delinq_lag1         0.012587
unemployment_lag1             0.012303
gdp_qoq_lag1                  0.008704
loan_to_asset_ratio_lag1      0.007179
vix_qoq_lag1                  0.006713
corp_bond_spread_lag1         0.005967
unemployment_diff_lag1        0.005606
sp500_qoq_lag1                0.002684
cons_sentiment_qoq_lag1       0.002520

--- Training Pooled XGBoost ---
  P

## Predict and evaluate

In [5]:

# --- 5. Prediction and Evaluation ---
print("\n" + "="*30); print(" Model Prediction & Evaluation "); print("="*30)
evaluation_results = {}
models_to_evaluate = [m for m, should_train in MODELS_TO_TRAIN.items() if should_train and trained_models.get(m) is not None]

evaluation_sets = {
    'train': (X_train, y_train_series),
    'val': (X_val, y_val[target_variable] if y_val is not None else None),
    'test': (X_test, y_test[target_variable] if y_test is not None else None)
}

for model_name in models_to_evaluate:
    model_obj = trained_models[model_name]
    pred_cols = training_columns.get(model_name) # Get columns model expects/used
    print(f"\n--- Evaluating {model_name} ---")
    evaluation_results[model_name] = {}
    for data_name, (X_data, y_actual) in evaluation_sets.items():
        if X_data is None or y_actual is None:
            print(f"  Skipping {data_name} set for {model_name}: Data missing.")
            evaluation_results[model_name][data_name] = {'R2': np.nan, 'MSE': np.nan, 'RMSE': np.nan, 'MAE': np.nan}
            continue
        # Generate predictions passing the required train_cols for OLS/FE
        predictions = run_predictions(model_obj, model_name, X_data, train_cols=pred_cols)
        # Evaluate
        metrics = evaluate_predictions(y_actual, predictions, model_name, data_name)
        evaluation_results[model_name][data_name] = metrics

# --- 6. Consolidate and Display Results ---
results_table = display_results(evaluation_results)
if results_table is not None:
    results_filename = os.path.join(SAVE_DIR, f"{timestamp_prefix}_evaluation_summary.csv")
    try: results_table.to_csv(results_filename); print(f"\nEvaluation summary saved to '{results_filename}'")
    except Exception as save_e: print(f"\nWarning: Could not save summary CSV: {save_e}")

# --- 7. Optional: Descriptive Statistics (Test Set) ---
print("\n" + "="*40); print(" Descriptive Statistics (Test Set) "); print("="*40)
y_test_series = y_test[target_variable] if y_test is not None else None
if X_test is not None and y_test_series is not None:
    print("\n--- Actual y_test Values ---")
    print(y_test_series.describe().to_string())
    for model_name in models_to_evaluate:
         model_obj = trained_models.get(model_name)
         if model_obj is None: continue
         print(f"\n--- {model_name} Predictions on Test Set ---")
         pred_cols = training_columns.get(model_name) # Get necessary cols for prediction
         predictions_test = run_predictions(model_obj, model_name, X_test, train_cols=pred_cols)
         if predictions_test is not None:
              predictions_aligned, _ = predictions_test.align(y_test_series, join='right')
              # Only describe non-NaN predictions
              print(predictions_aligned.dropna().describe().to_string())
         else: print("  Could not generate predictions for description.")
else: print("Test data unavailable, skipping descriptive statistics.")

# --- 8. Save models and best parameters ---
print("\n" + "="*40); print(" Saving Models and Artifacts "); print("="*40)
saved_count = 0
for model_name in models_to_evaluate:
    model_obj = trained_models.get(model_name)
    if model_obj is not None:
        save_model_artifacts(model_obj, model_name, best_params_all.get(model_name), SAVE_DIR, timestamp_prefix)
        saved_count += 1
if saved_count == 0: print("\nNo models were successfully trained and saved.")
else: print(f"\nSuccessfully saved artifacts for {saved_count} model(s).")

# --- End ---
end_time_float = time.time()
print("\n" + "="*40); print(f"Script finished at: {datetime.now()}");
print(f"Total execution time: {end_time_float - start_time:.2f} seconds."); print("="*40)


 Model Prediction & Evaluation 

--- Evaluating Pooled OLS ---
  Generating predictions for Pooled OLS...
  Generated 384 non-NaN Pooled OLS predictions.
  Evaluating Pooled OLS on train set...
    R2: 0.6958, RMSE: 0.0027, MAE: 0.0011 (384 obs)
  Generating predictions for Pooled OLS...
  Generated 28 non-NaN Pooled OLS predictions.
  Evaluating Pooled OLS on val set...
    R2: -2.4483, RMSE: 0.0012, MAE: 0.0009 (28 obs)
  Generating predictions for Pooled OLS...
  Generated 28 non-NaN Pooled OLS predictions.
  Evaluating Pooled OLS on test set...
    R2: -4.1109, RMSE: 0.0016, MAE: 0.0013 (28 obs)

--- Evaluating Bank FE ---
  Generating predictions for Bank FE...
  Generated 384 non-NaN Bank FE predictions.
  Evaluating Bank FE on train set...
    R2: -248.6858, RMSE: 0.0779, MAE: 0.0778 (384 obs)
  Generating predictions for Bank FE...
  Generated 28 non-NaN Bank FE predictions.
  Evaluating Bank FE on val set...
    R2: -15807.8788, RMSE: 0.0788, MAE: 0.0788 (28 obs)
  Generating

# Investigate

## OLS model summary

In [6]:
ols_model = trained_models.get('Pooled OLS')

ols_model.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                    roa   R-squared:                       0.696
Model:                            OLS   Adj. R-squared:                  0.682
Method:                 Least Squares   F-statistic:                     49.25
Date:                Sat, 17 May 2025   Prob (F-statistic):           1.38e-83
Time:                        18:17:52   Log-Likelihood:                 1723.6
No. Observations:                 384   AIC:                            -3411.
Df Residuals:                     366   BIC:                            -3340.
Df Model:                          17                                         
Covariance Type:            nonrobust                                         
==============================================================================================
                                 coef    std err          t      P>|t|      [0.025      0.975]
----------------------------------------------------------------------------------------------
const                         -0.0017      0.005     -0.331      0.741      -0.012       0.009
gdp_qoq_lag1                   0.0543      0.031      1.753      0.080      -0.007       0.115
cpi_qoq_lag1                   0.0750      0.046      1.649      0.100      -0.014       0.165
sp500_qoq_lag1                 0.0079      0.003      2.417      0.016       0.001       0.014
corp_bond_spread_lag1          0.0539      0.052      1.042      0.298      -0.048       0.155
cons_sentiment_qoq_lag1        0.0048      0.003      1.603      0.110      -0.001       0.011
unemployment_lag1             -0.0235      0.021     -1.133      0.258      -0.064       0.017
unemployment_diff_lag1         0.0918      0.044      2.070      0.039       0.005       0.179
household_delinq_lag1         -0.0299      0.018     -1.676      0.095      -0.065       0.005
household_delinq_diff_lag1    -0.1440      0.133     -1.085      0.279      -0.405       0.117
vix_qoq_lag1                   0.0021      0.001      2.419      0.016       0.000       0.004
spread_10y_3m_lag1             0.0390      0.028      1.414      0.158      -0.015       0.093
log_total_assets_lag1          0.0001      0.000      0.621      0.535      -0.000       0.001
deposit_ratio_lag1            -0.0040      0.002     -2.583      0.010      -0.007      -0.001
loan_to_asset_ratio_lag1       0.0048      0.002      2.262      0.024       0.001       0.009
equity_to_asset_ratio_lag1    -0.0059      0.007     -0.832      0.406      -0.020       0.008
trading_assets_ratio_lag1     -0.0041      0.006     -0.674      0.501      -0.016       0.008
roa_lag1                       0.5354      0.023     23.511      0.000       0.491       0.580
==============================================================================
Omnibus:                      343.621   Durbin-Watson:                   2.674
Prob(Omnibus):                  0.000   Jarque-Bera (JB):           163666.178
Skew:                           2.656   Prob(JB):                         0.00
Kurtosis:                     104.000   Cond. No.                     2.02e+04
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The condition number is large, 2.02e+04. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

## Multicollinearity

In [7]:
if False: 
    from statsmodels.stats.outliers_influence import variance_inflation_factor

    # Use the design matrix from OLS training (X_train_ols from train_pooled_ols function, ensure it includes constant)
    # Make sure X_train_ols exists from your training step or recalculate it
    # X_train_ols_for_vif = sm.add_constant(X_train, has_constant='add') # Or use the one saved if possible

    ols_fitted_cols = ols_model.model.exog_names # Get columns actually used in fit
    X_train_ols_fitted = ols_model.model.exog # Get the design matrix used in fit

    vif_data = pd.DataFrame()
    vif_data["feature"] = ols_fitted_cols
    # Calculate VIF - requires the exog matrix used in the fit
    vif_data["VIF"] = [variance_inflation_factor(X_train_ols_fitted, i) for i in range(X_train_ols_fitted.shape[1])]

    print("\nVariance Inflation Factors (VIF):")
    print(vif_data.sort_values('VIF', ascending=False))
    # Look for VIFs > 5 or 10

# Feature importance   

In [8]:
# For xgboost and rf, show feature importances
if 'Random Forest' in trained_models:
    rf_model = trained_models['Random Forest']
    if hasattr(rf_model, 'feature_importances_'):
        importances_rf = pd.Series(rf_model.feature_importances_, index=X_train.columns)
        print("\n--- Random Forest Feature Importances (Top 15) ---")
        print(importances_rf.nlargest(15).to_string())
if 'XGBoost' in trained_models:
    xgb_model = trained_models['XGBoost']
    if hasattr(xgb_model, 'feature_importances_'):
        importances_xgb = pd.Series(xgb_model.feature_importances_, index=X_train.columns)
        print("\n--- XGBoost Feature Importances (Top 15) ---")
        print(importances_xgb.nlargest(15).to_string())
# Note: Feature importances for OLS/FE are not directly available, but can be inferred from coefficients
# or VIF analysis.



--- Random Forest Feature Importances (Top 15) ---
roa_lag1                      0.728738
trading_assets_ratio_lag1     0.065668
equity_to_asset_ratio_lag1    0.056706
log_total_assets_lag1         0.045614
deposit_ratio_lag1            0.021562
household_delinq_diff_lag1    0.013678
household_delinq_lag1         0.012587
unemployment_lag1             0.012303
gdp_qoq_lag1                  0.008704
loan_to_asset_ratio_lag1      0.007179
vix_qoq_lag1                  0.006713
corp_bond_spread_lag1         0.005967
unemployment_diff_lag1        0.005606
sp500_qoq_lag1                0.002684
cons_sentiment_qoq_lag1       0.002520

--- XGBoost Feature Importances (Top 15) ---
deposit_ratio_lag1            0.466591
gdp_qoq_lag1                  0.165454
equity_to_asset_ratio_lag1    0.097067
roa_lag1                      0.080115
cpi_qoq_lag1                  0.049659
sp500_qoq_lag1                0.037519
trading_assets_ratio_lag1     0.023894
unemployment_diff_lag1        0.016345
log_t